# Modelos y Simulación 2024

## Trabajo Práctico Final:
## Hospital

### Alumnos:
- Ana Enzo
- Barea Matias
- Irigoyen Carlos
- Morillo Alen

### Profesores: Lic. Claudia López de Munain - Lic. Bruno Alfredo Pazos

In [1]:
from hospital import *

In [27]:
import random

quirofanos, ocupacion_quirofanos = inicializar_quirofanos(7)

quirofanos_list = list(quirofanos.keys())
print(quirofanos_list)
random.shuffle(quirofanos_list)
print(quirofanos_list)

['quirofano 1', 'quirofano 2', 'quirofano 3', 'quirofano 4', 'quirofano 5', 'quirofano 6', 'quirofano 7']
['quirofano 3', 'quirofano 1', 'quirofano 7', 'quirofano 4', 'quirofano 2', 'quirofano 5', 'quirofano 6']


In [2]:
''' TEST SIMULACION'''
estado_sistema = simulacion(2,210,4,12,0,35)
print(estado_sistema.keys())
print(f'Cantidad de quirofanos:', estado_sistema['cantidad de quirofanos:'],'\n',
        'Kits iniciales:', estado_sistema['kits_iniciales'], '\n',
        'Kits disponibles:', estado_sistema['kits_diarios_disponibles'], '\n',
        'kits utilizados por dia:', estado_sistema['kits_diarios_utilizados'], '\n',
        'llegada de pacientes por dia:', estado_sistema['llegadas_por_dia'], '\n',
        'reservas de quirofano por dia:', estado_sistema['reservas_por_dia'], '\n',
        'disponibilidad de camas por dia:', estado_sistema['disponibilidad_camas'], '\n',
        'ocupacion diaria quirofanos (en horas):', estado_sistema['ocupacion_diaria_quirofanos'], '\n',
        'cuanto esperan los pacientes por dia en ser operados:', estado_sistema['tiempo_espera_diario_quirofanos'] , '\n',
        'pacientes que solo se internaron:', estado_sistema['pacientes_sin_reserva_por_dia'], '\n',
        'pacientes rechazados por dia por ausencia de  camas :', estado_sistema['pacientes_rechazados_por_ausencia_camas'], '\n',
        'pacientes rechazados por dia por internacion agotada :', estado_sistema['pacientes_rechazados_por_internacion_agotada'], '\n'
        'cirugias concretadas por dia:', estado_sistema['cirugias_concretadas_por_dia'], '\n',
        'cirugias diarias reprogramadas por falta de insumos:', estado_sistema['cirugias_reprogramadas_por_insumos'], '\n',
        'cirugias diaria reprogramadas al otro dia por ocupacion del quirofano:', estado_sistema['cirugias_reprogramadas_por_tiempo'], '\n',
        'cirugias diaria reprogramadas al otro dia por cuota diria:', estado_sistema['cirugias_reprogramadas_por_cuota_diaria'], '\n',
        'Total de cirugias reprogramadas por falta insumos:', estado_sistema['total_cirugias_reprogramadas_por_insumos'], '\n',
        'Total de cirugias reprogramadas por falta de tiempo:', estado_sistema['total_cirugias_reprogramadas_por_tiempo'], '\n',
        'Total de cirugias reprogramadas por cuota diaria:', estado_sistema['total_cirugias_reprogramadas_por_cuota_diaria'], '\n',
        'Total cirugias concretadas:', estado_sistema['total_cirugias_concretadas'], '\n',
        'Total cirugias rechazadas:', estado_sistema['total_cirugias_rechazadas'], '\n',
        'Total pacientes rechazados por ausencia de camas:', estado_sistema['total_pacientes_rechazados_por_ausencia_camas'], '\n',
        'Total pacientes rechazados por internacion agotada:', estado_sistema['total_pacientes_rechazados_por_internacion_agotada'], '\n'
        'Promedio de espera de los pacientes: ', np.mean(estado_sistema['tiempo_espera_diario_quirofanos']), '\n',
        'Tiempo promedio de espera diario de los pacientes: ', (estado_sistema['tiempo_espera_diario_quirofanos']), '\n',
        'Longitud de espera de los pacientes: ', len(estado_sistema['tiempo_espera_diario_quirofanos']),


        )




dict_keys(['dias_simulacion', 'cantidad de quirofanos:', 'kits_iniciales', 'llegadas_por_dia', 'reservas_por_dia', 'disponibilidad_camas', 'pacientes_rechazados_por_dia', 'pacientes_rechazados_por_ausencia_camas', 'pacientes_rechazados_por_internacion_agotada', 'pacientes_sin_reserva_por_dia', 'kits_diarios_disponibles', 'kits_diarios_utilizados', 'ocupacion_diaria_quirofanos', 'tiempo_espera_diario_quirofanos', 'cirugias_rechazadas_por_dia', 'cirugias_concretadas_por_dia', 'cirugias_reprogramadas_por_insumos', 'cirugias_reprogramadas_por_tiempo', 'cirugias_reprogramadas_por_cuota_diaria', 'total_cirugias_reprogramadas_por_insumos', 'total_cirugias_reprogramadas_por_tiempo', 'total_cirugias_reprogramadas_por_cuota_diaria', 'total_cirugias_concretadas', 'total_cirugias_rechazadas', 'total_pacientes_rechazados_por_ausencia_camas', 'total_pacientes_rechazados_por_internacion_agotada'])
Cantidad de quirofanos: 4 
 Kits iniciales: 0 
 Kits disponibles: [3, 7, 10, 8, 12, 20, 20, 21, 31, 39, 

In [4]:

totales = calcular_porentaje_ocupacion_quirofanos(estado_sistema)
totales


{'quirofano 1': 92, 'quirofano 2': 92, 'quirofano 3': 92, 'quirofano 4': 92}

In [ ]:
def graficar(estado_sistema):
    # Días de la simulación
    dias_simulacion = estado_sistema['dias_simulacion']
    
    # Calcular el uso de kits mensualmente
    uso_kits_mensual = []
    for i in range(0, dias_simulacion, 30):  # Agrupar en meses (aproximadamente 30 días por mes)
        uso_mes = sum(estado_sistema['kits_diarios_utilizados'][i:i+30])
        uso_kits_mensual.append(uso_mes)
    
    # Meses para el gráfico
    meses = np.arange(1, len(uso_kits_mensual) + 1)
    
    # Graficar la utilización de kits mensualmente
    plt.figure(figsize=(10, 6))
    plt.bar(meses, uso_kits_mensual, color='skyblue')
    plt.xlabel('Mes de Simulación')
    plt.ylabel('Uso Mensual de Kits')
    plt.title('Uso Mensual de Kits a lo Largo de la Simulación')
    plt.xticks(meses)
    plt.grid(axis='y')
    plt.savefig('uso_mensual_kits.png', dpi=300)
    plt.show()